In [ ]:
# Create the Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Writing to Multiple Sinks")
    .config("spark.streaming.stopGracefullyOnShutdown", True)
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0')
    .config('spark.jars', '/home/jovyan/.ivy2/jars/org.postgresql_postgresql-42.2.20.jar')
    .config("spark.sql.shuffle.partitions", 8)
    .master("local[*]")
    .getOrCreate()
)

spark

In [ ]:
# Create the kafka_df to read from kafka

kafka_df = (
    spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "ed-kafka:29092")
    .option("subscribe", "device-data")
    .option("startingOffsets", "earliest")
    .load()
)

NameError: name 'spark' is not defined

In [ ]:
# View schema for raw kafka_df
kafka_df.printSchema()
#kafka_df.show()
#kafka_df.rdd.getNumPartitions()

In [ ]:
# Parse value from binay to string into kafka_json_df
from pyspark.sql.functions import expr

kafka_json_df = kafka_df.withColumn("value", expr("cast(value as string)"))

NameError: name 'kafka_df' is not defined

In [ ]:
# Schema of the Pyaload

from pyspark.sql.types import StringType, StructField, StructType, ArrayType, LongType

json_schema = (
    StructType(
    [StructField('customerId', StringType(), True),
    StructField('data', StructType(
        [StructField('devices',
                     ArrayType(StructType([
                        StructField('deviceId', StringType(), True),
                        StructField('measure', StringType(), True),
                        StructField('status', StringType(), True),
                        StructField('temperature', LongType(), True)
                    ]), True), True)
        ]), True),
    StructField('eventId', StringType(), True),
    StructField('eventOffset', LongType(), True),
    StructField('eventPublisher', StringType(), True),
    StructField('eventTime', StringType(), True)
    ])
)

In [ ]:
# Apply the schema to payload to read the data
from pyspark.sql.functions import from_json,col

streaming_df = kafka_json_df.withColumn("values_json", from_json(col("value"), json_schema)).selectExpr("values_json.*")


NameError: name 'kafka_json_df' is not defined

In [ ]:
# To the schema of the data, place a sample json file and change readStream to read
streaming_df.printSchema()
#streaming_df.show(truncate=False)

In [ ]:
# Lets explode the data as devices contains list/array of device reading
from pyspark.sql.functions import explode

exploded_df = streaming_df.withColumn("data_devices", explode("data.devices"))

NameError: name 'streaming_df' is not defined

In [ ]:
# Check the schema of the exploded_df, place a sample json file and change readStream to read
exploded_df.printSchema()
#exploded_df.show(truncate=False)

In [ ]:
# Flatten the exploded df
from pyspark.sql.functions import col

flattened_df = (
    exploded_df
    .drop("data")
    .withColumn("deviceId", col("data_devices.deviceId"))
    .withColumn("measure", col("data_devices.measure"))
    .withColumn("status", col("data_devices.status"))
    .withColumn("temperature", col("data_devices.temperature"))
    .drop("data_devices")
)

NameError: name 'exploded_df' is not defined

In [ ]:
# Check the schema of the flattened_df, place a sample json file and change readStream to read
flattened_df.printSchema()
#flattened_df.show(truncate=False)

In [ ]:
def device_data_output(df, batch_id):
  print('Batch ID '+ str(batch_id))
  df.write.format('parquet').mode('append').save('/home/rohit/data/device-data')

  # Write to JDBC Postgres
  (
      df.write
      .mode("append")
      .format("jdbc")
      .option("driver", "org.postgresql.Driver")
      .option("url", "jdbc:postgresql://postgres-db-1:5432/sqlpad")
      .option("dbtable", "device_data")
      .option("user", "sqlpad")
      .option("password", "sqlpad")
      .save()

  )
  # Diplay
  df.show()

SyntaxError: invalid syntax (<ipython-input-1-ef9b61c35576>, line 4)

In [ ]:
# Running foreachBatch
# Write the output to Multiple Sinks
(flattened_df
 .writeStream
 .foreachBatch(device_data_output)
 .trigger(processingTime='10 seconds')
 .option("checkpointLocation", "checkpoint_dir_kafka")
 .start()
 .awaitTermination())

NameError: name 'flattened_df' is not defined